# 量子线路编译

在 MindQuantum 中，量子线路编译由各种编译规则构成，这些规则会作用到量子线路对应的 DAG 图（Directed Acyclic Graph， 有向无环图）上，通过临近门融合、门替换的手段完成编译过程。

## DAG 图

在 MindQuantum 中，我们通过如下代码将一段量子线路转化为 DAG 图：

In [2]:
from mindquantum.algorithm.compiler import DAGCircuit
from mindquantum.core.circuit import Circuit

circ = Circuit().h(0).x(1, 0)
circ.svg()

In [3]:
dag_circ = DAGCircuit(circ)
dag_circ

在 DAG 图中，我们选择量子比特为头部节点，每个节点都有其子节点和父节点，例如上述量子线路中，`q0` 节点的子节点时 `H`，而 `H` 的父节点为 `q0` 且子节点为 `X(1 <-: 0)`.

In [7]:
print("All head node: ", dag_circ.head_node)
print("Child node of q0: ", dag_circ.head_node[0].child)
print("Father node of H(0): ", dag_circ.head_node[0].child[0].father)
print("Child node of H(0): ", dag_circ.head_node[0].child[0].child)

All head node:  {0: q0, 1: q1}
Child node of q0:  {0: H(0)}
Father node of H(0):  {0: q0}
Child node of H(0):  {0: X(1 <-: 0)}


通过上述的链式选择，我们可以选取到量子线路中的任意一个量子门。此外，DAG 图中的节点还有一个重要属性，`local` 指标，该指标表面节点是作用在哪些比特上，例如对于图中的 CNOT 门：

In [10]:
cnot_node = dag_circ.head_node[0].child[0].child[0]
print(cnot_node)
print(cnot_node.local)

X(1 <-: 0)
[1, 0]


此外，我们还可以直接将 DAG 复原为量子线路。

In [11]:
dag_circ.to_circuit().svg()

有了这些基本的 DAG 图结构，我们便可以在其上做量子编译。下面，我们介绍各种编译规则。

## 基本门分解规则 BasicDecompose

基本门分解规则会将多控制位的量子门、自定义的量子门分解为一些基本量子门，这些基本量子门集合为: `X`, `Y`, `Z`, `H`, `S`, `Sdag`, `T`, `TDag`, `SWAP`, `CNOT`, `RX`, `RY`, `RZ`, `U3`。

In [14]:
from mindquantum.algorithm.compiler import BasicDecompose
from mindquantum.core.gates import RX
circ = Circuit().x(2, [0, 1]) + RX('a').on(1, 0)
circ.svg()

In [15]:
dag_circ = DAGCircuit(circ)
rule = BasicDecompose()
rule.do(dag_circ)
new_circ = dag_circ.to_circuit()
new_circ.svg()

在 MindQuantum 中，我们调用编译规则的 `do` 函数来处理 DAG 图。至此，我们便完成了最基本的量子门分解。如上，我们还可发现，MindQuantum 支持对含参数门进行门分解。此外，MindQuantum 中的编译规则还支持调试功能，能够让你一步一步的看到哪些量子门发生了编译。我们通过调用编译规则的 `set_log_level` 来控制 log 信息，可选的 log_level 为：
- 0： 不打印任何调试信息，为默认值。
- 1： 打印粗略的调试信息
- 2： 打印全部的调试信息

In [17]:
dag_circ = DAGCircuit(circ)
rule = BasicDecompose()
rule.set_log_level(2)
rule.do(dag_circ)
new_circ.svg()

- Running BasicDecompose.
  BasicDecompose: gate X(2 <-: 0 1) will be compiled.
    - Running BasicDecompose.
    - BasicDecompose: nothing happened.
  BasicDecompose: gate RX(a|1 <-: 0) will be compiled.
    - Running BasicDecompose.
    - BasicDecompose: nothing happened.
- BasicDecompose: successfully compiled.


通过打印出的调试信息，我们发现，`BasicDecompose` 对 `X(2 <-: 0 1)` 和 `RX(a|1 <-: 0)` 都做了门分解。

## GateReplacer

该编译规则能够将指定模式的量子门替换成想要的量子线路。

例如对于如下的量子线路，我们可以将其中的 `CNOT` 全部换成 `H`、`CZ`和`H`的组合。

In [18]:
circ = Circuit().x(1, 0).h(1).x(1, 2)
circ.svg()

为此，我们构建如下的编译规则：

In [19]:
from mindquantum.algorithm.compiler import GateReplacer
from mindquantum.core.gates import X
rule = GateReplacer(X.on(1, 0), Circuit().h(1).z(1, 0).h(1))
rule.set_log_level(2)
rule

GateReplacer<
  q0: ──●──
        │
  q1: ──X── ->
  q0: ───────●───────
             │
  q1: ──H────Z────H──
>

In [20]:
dag_circ = DAGCircuit(circ)
rule.do(dag_circ)
new_circ = dag_circ.to_circuit()
new_circ.svg()

- Running GateReplacer.
  GateReplacer: gate X(1 <-: 2) will be replaced.
  GateReplacer: gate X(1 <-: 0) will be replaced.
- GateReplacer: successfully compiled.


由上面结果可知，除了第一个 CNOT 门，第二个作用顺序不一样的 CNOT 门我们也替换掉了。根据 `GateReplacer` 规则，我们在 MindQuantum 中实现了内置的将 CNOT 线路和 CZ 线路相互转化的编译维泽：

- `CXToCZ`：将 CNOT 门替换为 CZ 门。
- `CZToCX`：将 CZ 门替换成 CNOT 门。

根据上面的结果，我们发现，编译后产生了冗余的 H 门，这些 H 门可以删除掉，因此我们也实现了一个将一些量子门进行融合的编译规则，例如将两个 X 门融合为一个 identity，将两个 RX 门融合为一个 RX 门。

## FullyNeighborCanceler

该编译规则可以尝试将临近的多个门融合为一个量子门，或者直接删除掉（如果互为厄米共轭）。对于上述量子线路，我们接着利用该编译规则进行编译：

In [21]:
from mindquantum.algorithm.compiler import FullyNeighborCanceler

rule = FullyNeighborCanceler()
rule.set_all_log_level(2)
rule.do(dag_circ)
new_circ = dag_circ.to_circuit()
new_circ.svg()

- Running FullyNeighborCanceler: 1 child (SimpleNeighborCanceler, ).
  - Running SimpleNeighborCanceler.
      SimpleNeighborCanceler: merge H(1) and H(1).
  - SimpleNeighborCanceler: successfully compiled.
  FullyNeighborCanceler: state for each rule -> [True]
  - Running SimpleNeighborCanceler.
  - SimpleNeighborCanceler: nothing happened.
  FullyNeighborCanceler: state for each rule -> [False]
- FullyNeighborCanceler: successfully compiled.


由此，我们将对于的 H 门删除掉了。

在上述编译中，我们依次调用了两个编译规则：`GateReplacer` 和 `FullyNeighborCanceler`，为了更好将这些编译规则组织起来，我们实现了两个编译规则的容器，将多个编译规则组合成一个整体。

## SequentialCompiler

`SequentialCompiler` 是一个序列执行子编译规则的容器，其适用方法如下：

In [22]:
from mindquantum.algorithm.compiler import SequentialCompiler, CXToCZ



rules = SequentialCompiler([
    CXToCZ(),
    FullyNeighborCanceler(),
])
rules.set_all_log_level(2)
rules

SequentialCompiler<
  CXToCZ<
    GateReplacer<
      q0: ──X──
            │
      q1: ──●── ->
      q0: ──H────Z────H──
                 │
      q1: ───────●───────
    >
    GateReplacer<
      q0: ──X──
            │
      q1: ──●── ->
      q0: ──H────Z────H──
                 │
      q1: ───────●───────
    >
  >
  FullyNeighborCanceler<
    SimpleNeighborCanceler<>
  >
>

由上我们可知，rules 包含两条编译规则，`CXToCZ` 和 `FullyNeighborCanceler`。注意，我们内置的 `CXToCZ` 是一个能够将 MindQuantum 中的 `X.on(1, 0)` 和 `CNOT.on([1, 0])` 都转化为 CZ 线路的编译规则。

我们试着编译如下线路：

In [23]:
circ = circ = Circuit().x(1, 0).h(1).x(1, 2)
circ.svg()

In [25]:
dag_circ = DAGCircuit(circ)
rules.do(dag_circ)
new_circ = dag_circ.to_circuit()
new_circ.svg()

- Running SequentialCompiler: 2 child (CXToCZ, FullyNeighborCanceler, ).
  - Running CXToCZ: 2 child (GateReplacer, GateReplacer, ).
    - Running GateReplacer.
      GateReplacer: gate X(1 <-: 0) will be replaced.
      GateReplacer: gate X(1 <-: 2) will be replaced.
    - GateReplacer: successfully compiled.
    - Running GateReplacer.
    - GateReplacer: nothing happened.
    CXToCZ: state for each rule -> [True, False]
  - CXToCZ: successfully compiled.
  - Running FullyNeighborCanceler: 1 child (SimpleNeighborCanceler, ).
    - Running SimpleNeighborCanceler.
        SimpleNeighborCanceler: merge H(1) and H(1).
    - SimpleNeighborCanceler: successfully compiled.
    FullyNeighborCanceler: state for each rule -> [True]
    - Running SimpleNeighborCanceler.
    - SimpleNeighborCanceler: nothing happened.
    FullyNeighborCanceler: state for each rule -> [False]
  - FullyNeighborCanceler: successfully compiled.
  SequentialCompiler: state for each rule -> [True, True]
- SequentialCo

## KroneckerSeqCompiler

`KroneckerSeqCompiler` 也是一个由多个编译规则构成的容器，但是与 `SequentialCompiler` 不同的时，`KroneckerSeqCompiler`会重复执行内部的子编译规则，直到编译规则无法对量子线路产生效果为止。

# 总结

如上，便是 MindQuantum 中的量子线路编译实现。基于这些规则，我们也内置实现了一些适用于不同量子芯片编译规则，如只支持 CZ 门而不支持 CNOT 门的芯片，我们可以使用 `CZBasedChipCompiler` 来进行编译。

In [26]:
from mindquantum.algorithm.compiler import CZBasedChipCompiler

CZBasedChipCompiler()

CZBasedChipCompiler<
  BasicDecompose<>
  CXToCZ<
    GateReplacer<
      q0: ──X──
            │
      q1: ──●── ->
      q0: ──H────Z────H──
                 │
      q1: ───────●───────
    >
    GateReplacer<
      q0: ──X──
            │
      q1: ──●── ->
      q0: ──H────Z────H──
                 │
      q1: ───────●───────
    >
  >
  FullyNeighborCanceler<
    SimpleNeighborCanceler<>
  >
>

从结果我们可以发现，`CZBasedChipCompiler` 由三条编译规则构成：基本量子门分解规则，CX转化为CZ的编译规则，临近门消除规则。

# 附录

为了方便的使用这些编译规则进行量子线路编译，我们把编译规则的执行打包成了一个函数 `compile_circuit`，可以直接将给定的量子线路按照制定的规则进行编译。

In [28]:
from mindquantum.algorithm.compiler import compile_circuit

new_circ = compile_circuit(CZBasedChipCompiler().set_all_log_level(2), circ)
new_circ.svg()

- Running CZBasedChipCompiler: 3 child (BasicDecompose, CXToCZ, FullyNeighborCanceler, ).
  - Running BasicDecompose.
  - BasicDecompose: nothing happened.
  - Running CXToCZ: 2 child (GateReplacer, GateReplacer, ).
    - Running GateReplacer.
      GateReplacer: gate X(1 <-: 0) will be replaced.
      GateReplacer: gate X(1 <-: 2) will be replaced.
    - GateReplacer: successfully compiled.
    - Running GateReplacer.
    - GateReplacer: nothing happened.
    CXToCZ: state for each rule -> [True, False]
  - CXToCZ: successfully compiled.
  - Running FullyNeighborCanceler: 1 child (SimpleNeighborCanceler, ).
    - Running SimpleNeighborCanceler.
        SimpleNeighborCanceler: merge H(1) and H(1).
    - SimpleNeighborCanceler: successfully compiled.
    FullyNeighborCanceler: state for each rule -> [True]
    - Running SimpleNeighborCanceler.
    - SimpleNeighborCanceler: nothing happened.
    FullyNeighborCanceler: state for each rule -> [False]
  - FullyNeighborCanceler: successfully